In [2]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [4]:
REBUILD_DATA = True # set to true to one once, then back to false unless you want to change something in your training data.

class PNEUMONIA_DETECTOR():
    IMG_SIZE = 50
    NORMAL = "NORMAL"
    PNEUMONIA = "PNEUMONIA"
    dir = {
        "train": {
            NORMAL: "train/NORMAL",
            PNEUMONIA: "train/PNEUMONIA"
        },
        "test": {
            NORMAL: "test/NORMAL",
            PNEUMONIA: "test/PNEUMONIA"
        },
        "val": {
            NORMAL: "val/NORMAL",
            PNEUMONIA: "val/PNEUMONIA"
        },
    }

    LABELS = {NORMAL: 0, PNEUMONIA: 1}

    normalcount = 0
    pneumoniacount = 0

    def make_data(self, type):
        training_data = []
        for label, dir in self.dir[type].items():
            for f in tqdm(os.listdir(dir)):
                if "jpeg" in f:
                    try:
                        path = os.path.join(dir, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  # do something like print(np.eye(2)[1]), just makes one_hot
                        #print(np.eye(2)[self.LABELS[label]])
                        # print(len(self.training_data), len(self.training_data[-1][0]), len(self.training_data[-1][1]), self.training_data[-1])
                        if label == self.NORMAL:
                            self.normalcount += 1
                        elif label == self.PNEUMONIA:
                            self.pneumoniacount += 1


                    except Exception as e:
                        print(e)
                        pass
                        #print(label, f, str(e))

        training_data = np.array(training_data, dtype="object")

        np.random.shuffle(training_data)
        np.save(type+"_data.npy", training_data, allow_pickle=True)
        print('NORMAL:',self.normalcount)
        print('PNEUMONIA:',self.pneumoniacount)

if REBUILD_DATA:
    predictor = PNEUMONIA_DETECTOR()
    predictor.make_data("train")
    predictor.make_data("test")
    predictor.make_data("val")
    # print(dogsvcats.training_data.shape)

  0%|          | 0/1341 [00:00<?, ?it/s]

100%|██████████| 3875/3875 [01:42<00:00, 37.79it/s]


NORMAL: 1341
PNEUMONIA: 3875


100%|██████████| 390/390 [00:06<00:00, 58.56it/s]


NORMAL: 1575
PNEUMONIA: 4265


100%|██████████| 8/8 [00:00<00:00, 43.47it/s]

NORMAL: 1583
PNEUMONIA: 4273


In [ ]:
training_data = np.load("train_data.npy", allow_pickle=True)
testing_data = np.load("test_data.npy", allow_pickle=True)
validation_data = np.load("val_data.npy", allow_pickle=True)
print(len(training_data))
print(len(testing_data))
print(len(validation_data))

5216
624
16


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__() # just run the init of parent class (nn.Module)
        self.conv1 = nn.Conv2d(1, 32, 5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32, 64, 5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 kernel / window
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512) #flattening.
        self.fc2 = nn.Linear(512, 2) # 512 in, 2 out bc we're doing 2 classes (dog vs cat)

    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # bc this is our output layer. No activation here.
        return F.softmax(x, dim=1)

In [ ]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

/tmp/ipykernel_24500/4242875165.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)


In [ ]:
train_X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
train_X = train_X/255.0
train_y = torch.Tensor([i[1] for i in training_data])

In [ ]:
test_X = torch.Tensor([i[0] for i in testing_data]).view(-1,50,50)
test_X = test_X/255.0
test_Y = torch.Tensor([i[1] for i in testing_data])

In [ ]:
BATCH_SIZE = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev
        #print(f"{i}:{i+BATCH_SIZE}")
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+BATCH_SIZE]

        net.zero_grad()

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()    # Does the update

    print(f"Epoch: {epoch}. Loss: {loss}")